# Chain of Responsibility

Chain of objects who all get a chance to process a command. Procssing or Terminating the command.

## Example 1

In [4]:
class Engineer:
    def __init__(self, name, performance, salary, duration, happiness):
        self.name = name
        self.performance = performance
        self.salary = salary
        self.duration = duration
        self.happiness = happiness
    
    def __str__(self):
        return f"{self.name} ({self.performance}/{self.salary}/{self.duration}/{self.happiness})"
        
        
class Policy:
    def __init__(self, engineer):
        self.engineer = engineer
        self.next_policy = None
        
    def add_policy(self, policy):
        if self.next_policy:
            self.next_policy.add_policy(policy)
        else:
            self.next_policy = policy
    
    def handle(self):
        if self.next_policy:
            self.next_policy.handle()

class OvertimePolicy(Policy):
    def handle(self):
        print("Increase one hour working time.")
        self.engineer.performance *= 1.2
        self.engineer.duration *= 0.9
        self.engineer.happiness *= 0.9
        super().handle()

class RaisePolicy(Policy):
    def handle(self):
        print("Raise Salary!")
        self.engineer.performance *= 1.3
        self.engineer.salary *= 1.05
        self.engineer.duration *= 1.1
        self.engineer.happiness *= 1.1
        super().handle()

class NoPolicy(Policy):
    def handle(self):
        pass

In [6]:
john = Engineer('John', 100, 100, 100, 100)

root = Policy(john)
#root.add_policy(NoPolicy(john))
root.add_policy(RaisePolicy(john))
root.add_policy(OvertimePolicy(john))
root.handle()
print(john)


Raise Salary!
Increase one hour working time.
John (156.0/105.0/99.00000000000001/99.00000000000001)


In [3]:
root.handle()
print(john)

Increase one hour working time.
John (144.0/100/81.0/81.0)


## Example 2

event broker (observer)

In [24]:
from enum import Enum

class Event(list):
    def __call__(self, *args, **kwargs):
        for event in self:
            event(*args, **kwargs)

class Action(Enum):
    ATTACK = 1
    DEFENSE = 2

    
class Query:
    def __init__(self, name, action, value):
        self.value = value
        self.action = action
        self.name = name

class Booster:
    def __init__(self, battle, monster):
        self.monster = monster
        self.battle = battle
        self.battle.queries.append(self.handle)
    
    def handle(self, sender, query):
        pass
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.battle.queries.remove(self.handle)
        

class AttackBooster(Booster):
    def handle(self, sender, query):
        if sender.name == self.monster.name and query.action == Action.ATTACK:
            query.value *= 2
            
        
            
class Battle:
    def __init__(self):
        self.queries = Event()
    
    def perform_query(self, sender, query):
        self.queries(sender, query)
        
            
class Monster:
    def __init__(self, battle, name, attack, defense):
        self.name = name
        self.basic_attack = attack
        self.basic_defense = defense
        self.battle = battle
    
    @property
    def attack(self):
        q = Query(self.name, Action.ATTACK, self.basic_attack)
        self.battle.perform_query(self, q)
        return q.value

    @property
    def defense(self):
        q = Query(self.name, Action.DEFENSE, self.basic_defense)
        self.battle.perform_query(self, q)
        return q.value
    
    def __str__(self):
        return f'{self.name} ({self.attack}/{self.defense})'

In [25]:
battle = Battle()

slim = Monster(battle, 'Slim', 1, 1)
print(slim)
with AttackBooster(battle, slim):
    print(slim)

print(slim)

Slim (1/1)
Slim (2/1)
Slim (1/1)


## Example 3

Middleware


In [5]:
class Request:
    def __init__(self, environ):
        pass


class Middleware:
    def __init__(self, app):
        self.app = app
        
    def add_middleware(self, middleware):
        self.app = middleware(self.app)
    
    def process_request(self, request):
        pass
    
    def process_response(self, request, response):
        pass
    
    def handle_requests(request):
        self.process_request(request)
        response = self.app.handle_requests(request)
        self.process_response(request, response)
        return response
        
    def __call__(self, environ, start_response):
        request = Request(environ)
        response = self.handle_requests(request)
        return response(start_response)